In [8]:
import json
import pandas as pd
import pickle

In [27]:
with open('seasons.pickle') as f:
    seasons = pickle.load(f)
for year, season in seasons.iteritems():
    print year,
    season.load_scores()

2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018


In [10]:
with open('pd_data/name_fixes_all.json') as f:
    all_name_fixes = pickle.load(f)

In [ ]:
elt_df_list = []
comp_df_list = []
for season in (seasons['2017'], seasons['2018']):
    for event in season.events:
        for discipline in event.disciplines:
            for segment in discipline.segments:
                for scorecard in segment.scorecards:
                    for element in scorecard.elements:
                        elt_df_list.append({
                            'skater': all_name_fixes.get(scorecard.skater.name, scorecard.skater.name),
                            'country': scorecard.skater.country,
                            'segment_name': event.name + '_' + segment.name,
                            'season': season.champ_year,
                            'discipline': discipline.discipline.name,
                            'segment_rank': scorecard.rank,
                            'start_order': scorecard.starting_number,
                            'elt_number': element.number,
                            'elt_name': element.name,
                            'elt_info': element.info,
                            'elt_bv': element.base_value,
                            'elt_bonus': element.bonus,
                            'elt_finalgoe': element.goe,
                            'elt_goes': ','.join(map(str, element.goes)),
                            'elt_points': element.points
                        })
                    for comp in scorecard.components:
                        comp_df_list.append({
                            'skater': all_name_fixes.get(scorecard.skater.name, scorecard.skater.name),
                            'country': scorecard.skater.country,
                            'segment_name': event.name + '_' + segment.name,
                            'season': season.champ_year,
                            'discipline': discipline.discipline.name,
                            'segment_rank': scorecard.rank,
                            'start_order': scorecard.starting_number,
                            'comp_name': name_fixes_components.get(comp.name, comp.name),
                            'comp_scores': ','.join(map(str, comp.scores)),
                            'comp_points': comp.points
                        })

In [ ]:
elts_df = pd.DataFrame(elt_df_list)
comp_df = pd.DataFrame(comp_df_list)

In [ ]:
comp_df[comp_df.segment_name == 'gpusa2016_pairs_short']

In [12]:
def remove_mr_ms(judge):
    judge = judge.replace(r'\xa0', '')
    judge = judge.replace('Mr. ', '')
    judge = judge.replace('Mr ', '')
    judge = judge.replace('Ms. ', '')
    judge = judge.replace('Ms ', '')
    judge = judge.replace('Mrs. ', '')
    return judge

In [24]:
panels = {}  # map segment name to judging panel
judge_names = set()
for season in seasons.values():
    for event in season.events:
        for discipline in event.disciplines:
            for segment in discipline.segments:
                if 'Function' in segment.panel.officials:
                    del segment.panel.officials['Function']
                    print event, segment
                panel = {}
                for function, official in segment.panel.officials.iteritems():
                    name = remove_mr_ms(official.name)
                    name = all_name_fixes.get(name, name)
                    panel[function] = name
                    if len(name) < 5:
                        print event, segment, name
                    judge_names.add(name)
                panels[event.name + '_' + segment.name] = panel

gpusa2016 gpusa2016 pairs_short -
